In [151]:
cd "/home/sage/Documents/works/12. Developments/07. sage/02. IdIso/"

/home/sage/Documents/works/12. Developments/07. sage/02. IdIso


#  <font color="green">Implementation of IdealToIsogeny algorithm</font>

## <font color="green">0. Prime Search</font>

## <font color="green">1. Prerequisites</font>

### Load required libraries

In [152]:
load('quaternion_tools.sage')
load('isogeny.sage')
load('richelot.sage')
load('const.sage')

### Cornaccia algorithm

In [153]:
def cornacchia(d, m):
    try: 
        (x,y) = DiagonalQuadraticForm(QQ, [1,d]).solve(m)
        if not x.is_integer() or not y.is_integer() : raise Exception("No solution")
        return x, y
    except: raise Exception("No solution")

## <font color="green">2. Setting the lab environment</font>

### 1. Get a random ideal $I$ of norm $\ell^e$

In [154]:
I = GetRandomIdealElle()
assert(I.norm() == ell^6)

### 2. Extract the first ideal factor $I_1$
+ $I = I_1 \cdot I_2 \cdot ... \cdot I_k$ and $n(I_i) = \ell$

In [155]:
ellO0 = QA.ideal(O0.basis()).scale(ell)
I1 = AddIdeal(I, ellO0)

### 3. Compute corresponding isogeny $\phi_1$ to $I_1$
+ $ker(\phi_1) = ker(I_1)$

In [156]:
phi1_kernel_generator = left_O0_ideal_to_isogeny_kernel(I1)
phi1 = Isogeny(Fp(0), phi1_kernel_generator)
assert(I1.norm() == ell)

### 4. Get the codomain curve $E_1$ of $\phi_1$ and the dual isogeny $\hat{\phi}_1$

In [157]:
E1 = phi1.codomain_curve
phi1_dual = phi1.dual_isogeny()

In [158]:
cofactor = phi1_dual.domain_P.order() // 3^16
T = cofactor * phi1_dual.codomain_P
ellT = phi1.eval(T)
if ellT != phi1.degree * cofactor * phi1_dual.domain_P:
    phi1_dual.codomain_P *= -1
    phi1_dual.codomain_Q *= -1
    assert -ellT == phi1.degree * cofactor * phi1_dual.domain_P
else: assert ellT == phi1.degree * cofactor * phi1_dual.domain_P

T = cofactor * phi1.codomain_P
ellT = phi1_dual.eval(T)
assert ellT == phi1_dual.degree * cofactor * phi1.domain_P

In [159]:
print(ellT)
print(phi1.degree * cofactor * phi1_dual.domain_P)

(362750233691548140798995600727614572686366654846789*Fp2_i^2 + 350789259401479485449374434308266133683402966752001 : 211974081782683066240323151608656360520265412250364*Fp2_i^3 + 112976886149074134416097968040078717520779001172653*Fp2_i : 1)
(19393888032629786834706910273510803357420530885399*Fp2_i^2 + 53359580631560546077816198963595015852988586891890 : 117279626687547657424506532914207390915449117221240*Fp2_i^3 + 84706836072486172756874772705305850077387248615495*Fp2_i : 1)


### 5. Compute the right order $\mathcal{O}_1$ of $I_1$
+ $\mathcal{O}_1 \cong End(E_1)$

In [202]:
O1 = I1.right_order()

### 6. Extract the second ideal factor $I_2$
+ $\bar{I_1} I \cap \ell \mathcal{O}_1 = I_2$

In [203]:
I2 = I1.conjugate() * I
I2 = I2.scale(1/ell)
ellO1 = QA.ideal(O1.basis()).scale(ell)
I2 = AddIdeal(I2, ellO1)

assert(I2.norm() == ell and I2.left_order() == O1)

## <font color="green">3. Experiments</font>

+ Compute the generator $\alpha_2$ of $I_2$ such that $I_2 = \alpha_2 \mathcal{O_1} + \ell \mathcal{O_1}$
+ Hopefully $n(\alpha_2)$ is as small as possible
+ Note that $\alpha_2 \in I_2 \subset \mathcal{O}_1$

In [162]:
I2_generator = ideal_generator(I2)

+ Compute the reduced basis $\{o_{11}, o_{12}, o_{13}, o_{14}\}$ of $\mathcal{O_1}$

In [163]:
O1_basis = GetReducedBasis(O1.basis())

+ Try to find the $\beta \in \mathcal{O}_1$ such that $n(\alpha_2) + n(\beta) = 2^{e_1}\cdot 3^{e_2} | \sqrt{\#E_1(\mathbb{F}_{p^k})}$
+ $\mathrm{ord}_{2}(\#E_1(\mathbb{F}_{p^k})) \approx \mathrm{ord}_2(\#E_1(\mathbb{F}_{p^2}))=2 \times \mathrm{ord}_2(p+1)$ regardless of the extension size $k$
    + So it must be satisfied that $2^e < (p+1)$
+ Find the coefficients $\{c_1, c_2, c_3, c_4\}$ such that $n(\beta) = n(\sum_{i}{c_i o_{1i}}) = 2^e - n(\alpha_2)$

In [164]:
import random
I2_basis = GetReducedBasis(I2.basis())
I2_generator = I2_basis[2]
e1 = 148
e2 = 16
gram = matrix([[1,0,0,0],[0,1,0,0],[0,0,prime,0],[0,0,0,prime]])
M_O1 = matrix([vector(v.coefficient_tuple()) for v in O1_basis])
G = M_O1 * gram * M_O1.transpose()

x,y = 0, 0
k = 100
for i in range(k^2):
    I2_gen = I2_generator + random.randint(-k, k) * I2_basis[1] + random.randint(-k, k) * I2_basis[0]
    if gcd(I2_gen.reduced_norm() ,6) != 1: continue
    target = 2^e1*3^e2 - I2_gen.reduced_norm()
    assert target > 0
    if kronecker(-G[1][1], target) != 1: continue
    try:
        x,y = cornacchia(G[1][1], target)
        break
    except:
        print("no solution of cornacchia algorithm")
beta = O1_basis[0]*x + O1_basis[1]*y
print((beta.reduced_norm() + I2_gen.reduced_norm()).factor())
assert gcd(beta.reduced_norm(), I2_gen.reduced_norm()) == 1

no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
no solution of cornacchia algorithm
2^148 * 3^16


In [165]:
N = 2^e1*3^e2
N_alpha = I2_gen.reduced_norm()
N_beta = beta.reduced_norm()
x_value = Integer(mod(N_beta, N)^-1)

In [166]:
P1, Q1 = phi1.codomain_curve.gens()
cofactor = Integer((prime^2 - 1) / N)
P1, Q1 = cofactor*P1, cofactor*Q1
E1.is_on_curve(P1[0], P1[1])

True

In [167]:
load('isogeny.sage')
psi_P1 = eval_endomorphism(x_value * I2_gen * beta, P1, phi1, phi1_dual)
psi_Q1 = eval_endomorphism(x_value * I2_gen * beta, Q1, phi1, phi1_dual)

(-7815875918828717155939129214263518379603823795150873276163660284588277964080234765, -2220381682337876288311106669088578353382715389120267563798303109715070869225966603, 60129113454054068752346367638309679255725017276692681251815610532684339276078, 1373318059240740907631034951502314569950945651840298867340008449481028010292485)
(-7815875918828717155939129214263518379603823795150873276163660284588277964080234765, -2220381682337876288311106669088578353382715389120267563798303109715070869225966603, 60129113454054068752346367638309679255725017276692681251815610532684339276078, 1373318059240740907631034951502314569950945651840298867340008449481028010292485)


In [168]:
points = [P1, Q1, psi_P1, psi_Q1]
points = [E1((e[0], e[1])) for e in points]
assert points[0].weil_pairing(points[1], N)*points[2].weil_pairing(points[3], N) == 1
ell_cofactor = sqrt(phi1.codomain_curve.cardinality()) // ell
[ell_P, ell_Q] = [point * ell_cofactor for point in E1.gens()]
points = points + [ell_P, ell_Q]


In [169]:
load('../01. jacobian_isogeny/22_isogeny.sage')
points = [(points[0], points[2]), (points[1], points[3]), (points[4], E1(0)), (points[5], E1(0))]
a, b = 148, 16
kernel2 = [(3^b*2^(a-1)*D[0], 3^b*2^(a-1)*D[1]) for D in [points[0], points[1]]]
E_start = E1
h, points, isog = Gluing22(E_start, E_start, kernel2, eval_points = points)

In [170]:
kernel22 = [3^b * 2 * D for D in points[:2]]
j, points = isogeny_22(kernel22, points, a-2)

In [171]:
load('../01. jacobian_isogeny/33_isogeny.sage')
kernel33 = [2 * D for D in points[:2]]
K = Fp
j, points = isogeny_33(kernel33, points, b)

In [172]:
G1 = points[0][0]
G2 = points[1][0]
h = j.curve().hyperelliptic_polynomials()[0]
G3, r3 = h.quo_rem(G1*G2)
assert r3 == 0
delta = Matrix(G.padded_list(3) for G in (G1, G2, G3))
assert delta.determinant() == 0

In [173]:
load('../01. jacobian_isogeny/22_isogeny.sage')
kernel22 = [D for D in points[:2]]
prodE, eval_points = Splitting22(kernel22, [points[2], points[3]])

In [174]:
eval_points[1][1] * 5

(0 : 1 : 0)

In [175]:
assert E1.j_invariant() in [T.j_invariant() for T in prodE]
P, Q = E1(0), E1(0)
if prodE[0].j_invariant() == E1.j_invariant():
    P, Q = eval_points[0][0], eval_points[1][0]
else:
    P, Q = eval_points[0][1], eval_points[1][1]

second_kernel = E1(0)
if P.is_zero(): second_kernel = ell_Q
elif Q.is_zero(): second_kernel = ell_P
else:
    assert P.weil_pairing(Q, P.order()) == 1

    r = P.discrete_log(Q)
    second_kernel = ell_P * r - ell_Q
print("second kernel :", second_kernel)


second kernel : (56004228921922556418156105857225276178464519711839*Fp2_i^2 + 225095929295850743235266709851260428747426821946082 : 383778502041296658666916190570502519238220336948971*Fp2_i^2 + 9330114020321753822818836852433738044600044132628 : 1)


In [198]:
O2 = I2.right_order()

In [206]:
I3 = I2.conjugate() * I1.conjugate() * I
I3 = I3.scale(ell^-2)
ellO2 = QA.ideal(O2.basis()).scale(ell)
I3 = AddIdeal(I3, ellO1)
assert I3.norm() == ell and I3.left_order() == O2

AssertionError: 

In [176]:
phi2 = Isogeny(phi1.codomain_coeff, second_kernel)

In [182]:
E2 = phi2.codomain_curve
phi2_dual = phi2.dual_isogeny()

In [183]:
cofactor = phi2_dual.domain_P.order() // 3^16
T = cofactor * phi2_dual.codomain_P
ellT = phi2.eval(T)
if ellT != phi2.degree * cofactor * phi2_dual.domain_P:
    phi2_dual.codomain_P *= -1
    phi2_dual.codomain_Q *= -1
    assert -ellT == phi2.degree * cofactor * phi2_dual.domain_P
else: assert ellT == phi2.degree * cofactor * phi2_dual.domain_P

T = cofactor * phi2.codomain_P
ellT = phi2_dual.eval(T)
assert ellT == phi2_dual.degree * cofactor * phi2.domain_P

In [192]:
O2 = I2.right_order()